In [1]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)

1.3.0


## Load data

In [4]:
DATASET_PATH = "./dataset/UCI HAR Dataset/"

In [5]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

In [6]:
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        with open(signal_type_path, "r") as f:
            X_signals.append(
                [np.array(serie, dtype=np.float32)
                    for serie in [row.replace('  ', ' ').strip().split(' ') for row in f]]
            )
    return np.transpose(X_signals, (1, 2, 0))

def load_y(y_path):

    # Read dataset from disk, dealing with text file's syntax
    with open(y_path, "r") as f:
        y = np.array(
            [elem for elem in [
                row.replace('  ', ' ').strip().split(' ') for row in f
            ]], 
            dtype=np.int32
        )
            
    y = y.reshape(-1,)
    # Substract 1 to each output class for friendly 0-based indexing 
    return y - 1

In [7]:
X_train_signals_paths = [
    DATASET_PATH + "train/Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + "test/Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

In [8]:
har_X_train = load_X(X_train_signals_paths)
har_X_test = load_X(X_test_signals_paths)

In [9]:
y_train_path = DATASET_PATH + "train/y_train.txt"
y_test_path = DATASET_PATH + "test/y_test.txt"

In [10]:
har_y_train = load_y(y_train_path)
har_y_test = load_y(y_test_path)

## Prepare data

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(har_X_train, har_y_train, random_state=42)

In [12]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

In [13]:
X_train = np.transpose(X_train, (0, 2, 1))
X_valid = np.transpose(X_valid, (0, 2, 1))
har_X_test = np.transpose(har_X_test, (0, 2, 1))

In [14]:
X_train.shape, y_train.shape

((5514, 9, 128), (5514,))

# Build RNN model

## Hyperparameters

In [15]:
batch_size = 200
n_epochs = 200
learning_rate = 0.003

seq_len = 128
n_channels = 9
n_neurons = 100
n_layers = 3

# outputs
n_classes = 6

In [16]:
n_batches = X_train.shape[0] // batch_size

## a simple Basic RNN model

In [17]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [18]:
with tf.name_scope("rnn"):
    
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                             activation=tf.nn.relu)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
    
    # only one RNN cell, the shape of states is (None, n_neurons)
    #states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states, n_classes)

In [19]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [20]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [21]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [22]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [23]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_first_rnn_model.ckpt")

Epoch:  0 Loss: 0.507086 Valid accuracy:  0.781828
Epoch:  10 Loss: 0.203461 Valid accuracy:  0.926551
Epoch:  20 Loss: 0.189832 Valid accuracy:  0.926007
Epoch:  30 Loss: 0.210398 Valid accuracy:  0.927639
Epoch:  40 Loss: 0.147995 Valid accuracy:  0.945593
Epoch:  50 Loss: 0.161578 Valid accuracy:  0.95049
Epoch:  60 Loss: 0.205865 Valid accuracy:  0.945593
Epoch:  70 Loss: 0.174509 Valid accuracy:  0.949946
Epoch:  80 Loss: 0.187571 Valid accuracy:  0.951578
Epoch:  90 Loss: 0.31176 Valid accuracy:  0.92111
Epoch:  100 Loss: 0.159375 Valid accuracy:  0.959195
Epoch:  110 Loss: 0.208423 Valid accuracy:  0.960283
Epoch:  120 Loss: 0.191918 Valid accuracy:  0.955386
Epoch:  130 Loss: 0.220724 Valid accuracy:  0.960283
Epoch:  140 Loss: 0.27311 Valid accuracy:  0.960283
Epoch:  150 Loss: 0.300344 Valid accuracy:  0.959739
Epoch:  160 Loss: 0.316955 Valid accuracy:  0.957563
Epoch:  170 Loss: 0.330345 Valid accuracy:  0.958107
Epoch:  180 Loss: 0.342931 Valid accuracy:  0.957018
Epoch:  

In [24]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_first_rnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_first_rnn_model.ckpt
valid Accuracy:  0.955386
Test Accuracy:  0.908721


## Basic RNN model with dropout

In [25]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [26]:
with tf.name_scope("rnn"):
    
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                             activation=tf.nn.relu)
    drop_basic_cell = tf.contrib.rnn.DropoutWrapper(basic_cell, output_keep_prob=0.8)
    outputs, states = tf.nn.dynamic_rnn(drop_basic_cell, X, dtype=tf.float32)
    
    # only one RNN cell, the shape of states is (None, n_neurons)
    #states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(states, n_classes)

In [27]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [28]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [29]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [30]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [31]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_rnn_dropout_model.ckpt")

Epoch:  0 Loss: 0.514964 Valid accuracy:  0.774211
Epoch:  10 Loss: 0.182779 Valid accuracy:  0.939064
Epoch:  20 Loss: 0.142929 Valid accuracy:  0.949946
Epoch:  30 Loss: 0.141336 Valid accuracy:  0.947225
Epoch:  40 Loss: 0.180644 Valid accuracy:  0.945593
Epoch:  50 Loss: 0.214971 Valid accuracy:  0.935256
Epoch:  60 Loss: 0.173295 Valid accuracy:  0.955386
Epoch:  70 Loss: 0.188844 Valid accuracy:  0.943417
Epoch:  80 Loss: 0.190031 Valid accuracy:  0.957018
Epoch:  90 Loss: 0.231158 Valid accuracy:  0.956474
Epoch:  100 Loss: 0.211775 Valid accuracy:  0.961371
Epoch:  110 Loss: 0.240163 Valid accuracy:  0.961371
Epoch:  120 Loss: 0.265905 Valid accuracy:  0.960827
Epoch:  130 Loss: 0.198752 Valid accuracy:  0.957563
Epoch:  140 Loss: 0.289762 Valid accuracy:  0.944505
Epoch:  150 Loss: 0.255943 Valid accuracy:  0.957018
Epoch:  160 Loss: 0.309293 Valid accuracy:  0.939064
Epoch:  170 Loss: 0.253106 Valid accuracy:  0.963003
Epoch:  180 Loss: 0.230918 Valid accuracy:  0.967356
Epoc

In [32]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_rnn_dropout_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_rnn_dropout_model.ckpt
valid Accuracy:  0.96518
Test Accuracy:  0.920937


# Three Basic RNN layers with  early stopping

In [33]:
# maximum number of epoch
n_epochs=600

In [34]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [35]:
with tf.name_scope("rnn"):
    

    cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
                 for _ in range(n_layers)]
    
    multi_cells = tf.contrib.rnn.MultiRNNCell(cells)
   
    outputs, states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [36]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [37]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [38]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [39]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [40]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [41]:
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress=150

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
    
        if epoch % 5 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=4) < best_loss:
                saver.save(sess, "./model/har_basic_rnn_early_stopping_model.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    break

Epoch:  0 Loss:  0.47102 Valid accuracy:  0.793254
Epoch:  5 Loss:  0.201861 Valid accuracy:  0.927639
Epoch:  10 Loss:  0.16398 Valid accuracy:  0.945049
Epoch:  15 Loss:  0.172413 Valid accuracy:  0.945049
Epoch:  20 Loss:  0.186989 Valid accuracy:  0.940152
Epoch:  25 Loss:  0.113493 Valid accuracy:  0.96518
Epoch:  30 Loss:  0.168028 Valid accuracy:  0.958107
Epoch:  35 Loss:  0.142219 Valid accuracy:  0.960827
Epoch:  40 Loss:  0.130022 Valid accuracy:  0.967356
Epoch:  45 Loss:  0.216629 Valid accuracy:  0.952666
Epoch:  50 Loss:  0.195717 Valid accuracy:  0.95593
Epoch:  55 Loss:  0.20686 Valid accuracy:  0.954842
Epoch:  60 Loss:  0.181368 Valid accuracy:  0.965724
Epoch:  65 Loss:  0.188134 Valid accuracy:  0.968444
Epoch:  70 Loss:  0.207753 Valid accuracy:  0.964635
Epoch:  75 Loss:  0.224124 Valid accuracy:  0.964091
Epoch:  80 Loss:  0.232567 Valid accuracy:  0.964091
Epoch:  85 Loss:  0.23896 Valid accuracy:  0.963547
Epoch:  90 Loss:  0.244448 Valid accuracy:  0.963547
E

In [42]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_basic_rnn_early_stopping_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_basic_rnn_early_stopping_model.ckpt
valid Accuracy:  0.96518
Test Accuracy:  0.917883


## Bi-directional Basic RNN

In [43]:
n_epochs=200

In [44]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [45]:
with tf.name_scope("rnn"):
    
    fw_cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                                 activation=tf.nn.relu)
                         for _ in range(n_layers)]
    
    bw_cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                                 activation=tf.nn.relu)
                         for _ in range(n_layers)]
    multi_fw_cells = tf.contrib.rnn.MultiRNNCell(fw_cells)
    multi_bw_cells = tf.contrib.rnn.MultiRNNCell(bw_cells)
        
    outputs, states = tf.nn.bidirectional_dynamic_rnn(multi_fw_cells, multi_bw_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1][-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [46]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [47]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [48]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [49]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [50]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [51]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
             
        if epoch % 10 == 0:
            
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_rnn_model.ckpt")

Epoch:  0 Loss: 0.544632 Valid accuracy:  0.767138
Epoch:  10 Loss: 0.1603 Valid accuracy:  0.931447
Epoch:  20 Loss: 0.160306 Valid accuracy:  0.947225
Epoch:  30 Loss: 0.163517 Valid accuracy:  0.95321
Epoch:  40 Loss: 0.165518 Valid accuracy:  0.955386
Epoch:  50 Loss: 0.190788 Valid accuracy:  0.94124
Epoch:  60 Loss: 0.1744 Valid accuracy:  0.960283
Epoch:  70 Loss: 0.160103 Valid accuracy:  0.959195
Epoch:  80 Loss: 0.19295 Valid accuracy:  0.964635
Epoch:  90 Loss: 0.215903 Valid accuracy:  0.96518
Epoch:  100 Loss: 0.157172 Valid accuracy:  0.96518
Epoch:  110 Loss: 0.221274 Valid accuracy:  0.9679
Epoch:  120 Loss: 0.24104 Valid accuracy:  0.966268
Epoch:  130 Loss: 0.253817 Valid accuracy:  0.965724
Epoch:  140 Loss: 0.263601 Valid accuracy:  0.96518
Epoch:  150 Loss: 0.272024 Valid accuracy:  0.96518
Epoch:  160 Loss: 0.279599 Valid accuracy:  0.965724
Epoch:  170 Loss: 0.28653 Valid accuracy:  0.965724
Epoch:  180 Loss: 0.293022 Valid accuracy:  0.965724
Epoch:  190 Loss: 0

In [52]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_rnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_rnn_model.ckpt
valid Accuracy:  0.965724
Test Accuracy:  0.900916


# Bi-directional Basic RNN with dropout and early stopping

In [53]:
n_epochs=600

In [54]:
tf.reset_default_graph()
tf.set_random_seed(73)
np.random.seed(73)

X = tf.placeholder(tf.float32, (None, n_channels, seq_len), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [55]:
with tf.name_scope("rnn"):
    
    fw_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
                                             output_keep_prob=0.8)
                   for _ in range(n_layers)]
    bw_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
                                             output_keep_prob=0.8)
                   for _ in range(n_layers)]
    
    multi_fw_cells = tf.contrib.rnn.MultiRNNCell(fw_cells)
    multi_bw_cells = tf.contrib.rnn.MultiRNNCell(bw_cells)
    
    outputs, states = tf.nn.bidirectional_dynamic_rnn(multi_fw_cells, multi_bw_cells, X, dtype=tf.float32)
    top_layer_h_state = states[-1][-1]
    states_concat = tf.concat(axis=1, values=states)
    logits = tf.layers.dense(top_layer_h_state, n_classes)

In [56]:
top_layer_h_state.shape

TensorShape([Dimension(None), Dimension(100)])

In [57]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [58]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [59]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [60]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [61]:
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress=100

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for i in range(n_batches):
            X_batch = X_train[i * batch_size:(i+1) * batch_size]
            y_batch = y_train[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
    
        if epoch % 5 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=4) < best_loss:
                saver.save(sess, "./model/har_rnn_early_stopping_model.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    break

Epoch:  0 Loss:  0.660713 Valid accuracy:  0.647987
Epoch:  5 Loss:  0.231613 Valid accuracy:  0.912405
Epoch:  10 Loss:  0.161311 Valid accuracy:  0.940696
Epoch:  15 Loss:  0.202107 Valid accuracy:  0.934168
Epoch:  20 Loss:  0.156147 Valid accuracy:  0.946137
Epoch:  25 Loss:  0.144424 Valid accuracy:  0.951034
Epoch:  30 Loss:  0.22227 Valid accuracy:  0.933623
Epoch:  35 Loss:  0.124664 Valid accuracy:  0.952666
Epoch:  40 Loss:  0.146628 Valid accuracy:  0.952122
Epoch:  45 Loss:  0.184355 Valid accuracy:  0.952122
Epoch:  50 Loss:  0.179646 Valid accuracy:  0.946681
Epoch:  55 Loss:  0.140168 Valid accuracy:  0.963003
Epoch:  60 Loss:  0.200739 Valid accuracy:  0.952122
Epoch:  65 Loss:  0.164068 Valid accuracy:  0.953754
Epoch:  70 Loss:  0.165173 Valid accuracy:  0.958651
Epoch:  75 Loss:  0.196786 Valid accuracy:  0.952122
Epoch:  80 Loss:  0.137225 Valid accuracy:  0.960827
Epoch:  85 Loss:  0.226055 Valid accuracy:  0.949401
Epoch:  90 Loss:  0.202783 Valid accuracy:  0.946

In [62]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_rnn_early_stopping_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_rnn_early_stopping_model.ckpt
valid Accuracy:  0.970076
Test Accuracy:  0.904649
